# EJERCICIO

Usando el df_titanic, coger las columans numéricas excepto "PassengerId, y usar también la columna "Sex" Para datos precitores.

Usar la columna "Survived" como objetivo.

Probar por separado los algoritmos SupportVectorMachine (kernel lineal), DecisionTreeClassifier (max_depth -> 2) y KNeighborsClassifier (25 n_neighbors), ver los

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
df_titanic = pd.read_csv("./titanic1.csv")

df_titanic["Sex_num"] = df_titanic["Sex"].map({"male": 0, "female": 1}) # Creamos Columna Sex_num. Sex es texto, pero sex_num la hemos 
df_num = df_titanic.select_dtypes(include="number") # Nos quedamos con todos los datos númericos
df_num.drop(columns="PassengerId", inplace = True) # Modifica directamete el DF. # Vamos a quietar PassengerId (que también es numérica) por que es RUIDO, no nos aporta nada

df_num.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_num
0,0,3,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1


In [31]:
df_num.info() # Observamos que una de las columnas tiene menos datos, ergo tiene filas NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Sex_num   891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [17]:
X = df_num.drop(columns="Survived")
y = df_num["Survived"] # Variable objetivo

In [18]:
X["Age"].fillna(X["Age"].mean(), inplace = True)
df_num.head(2)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_num
0,0,3,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 40)

### Vamos a ver el algoritmo Support Vector Machine

In [20]:
SVM_clf = SVC(kernel="linear", random_state = 40)
SVM_clf.fit(X_train, y_train)
scoreSVM = SVM_clf.score(X_test, y_test)
print(scoreSVM)

0.8044692737430168


### Vamos a ver el algoritmo DecisionTreeClassifier

In [21]:
DTC_clf = DecisionTreeClassifier(max_depth = 2, random_state = 40)
DTC_clf.fit(X_train, y_train)
scoreDTC = DTC_clf.score(X_test, y_test)
print(scoreDTC)

0.7988826815642458


### Vamos a ver el algoritmo KNeighborsClassifier

In [22]:
KNN_clf = KNeighborsClassifier(n_neighbors=25) # Lo vamos hacer sin parametros, aunque nosmalmente SÍ PONDREMOS PARÁMETROS
KNN_clf.fit(X_train, y_train)
scoreKNN = KNN_clf.score(X_test, y_test)
print(scoreKNN)

0.7262569832402235


---

### Vamos a ver el algoritmo StackingClassifier

In [23]:
estimadores = [("SVC", SVC(kernel="linear", random_state = 40)), ("cart", DecisionTreeClassifier(max_depth = 2, random_state = 40))]
stacking_clf = StackingClassifier(estimators = estimadores, final_estimator = KNeighborsClassifier(n_neighbors=25))

In [25]:
stacking_clf.fit(X_train, y_train)
scoreStack = stacking_clf.score(X_test, y_test)
print(scoreStack) # LO NORMAL es que mejore

0.8044692737430168


In [26]:
estimadores = [("knn", KNeighborsClassifier(n_neighbors=25)), ("cart", DecisionTreeClassifier(max_depth = 2, random_state = 40))]
stacking_clf = StackingClassifier(estimators = estimadores, final_estimator = SVC(kernel="linear", random_state = 40))

In [27]:
stacking_clf.fit(X_train, y_train)
scoreStack = stacking_clf.score(X_test, y_test)
print(scoreStack) # LO NORMAL es que mejore

0.7988826815642458


In [28]:
estimadores = [("knn", KNeighborsClassifier(n_neighbors=25)), ("svc", SVC(kernel="linear", random_state = 40))]
stacking_clf = StackingClassifier(estimators = estimadores, final_estimator = DecisionTreeClassifier(max_depth = 2, random_state = 40))

In [29]:
stacking_clf.fit(X_train, y_train)
scoreStack = stacking_clf.score(X_test, y_test)
print(scoreStack) # LO NORMAL es que mejore

0.8212290502793296


---
---